In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# 이미지에 들어가는 한글을 제대로 보기 위해 한글 폰트 적용
import platform
font_dict = {
    'Linux': 'Noto Sans CJK KR',
    'Darwin': 'Apple SD Gothic Neo', # macOS
    'Windows': 'Malgun Gothic' # MS-Windows
}
try:
    mpl.rc('font', family=font_dict[platform.system()])
except:
    pass
mpl.rc('axes', unicode_minus=False)

%matplotlib inline

# [All about 따릉이 EDA, 3편] 따릉이 이용량 더 자세하게 살펴보기 by 흠시

**출처:** https://dailyheumsi.tistory.com/88

**데이터:** 서울특별시 공공자전거 대여이력 정보 @[서울 열린데이터 광장](https://data.seoul.go.kr)
  - 자전거 이동경로에 대한 데이터 분석이 가능하도록 년도별, 대여소별, 자전거별 대여이력 원천 데이터를 제공
  - https://data.seoul.go.kr/dataList/OA-15182/F/1/datasetView.do
  - `서울특별시 공공자전거 대여정보_201905.csv` (용량 299.3MB, 수정일 2019.06.15)

> [흠시] 이전 글에 이어, 이번에는 좀 더 딥하게 탐색해보기로 한다.  
이전에는 15년 9월 - 18년 11월의 일일 데이터를 다루었다면,   
이번에는**가장 최근에 이용량이 많았던** **19년 5월 시간별 데이터**만 보기로 한다. 

> [흠시] 이제부터는 요일, 시간 단위의 데이터를 볼 수 있다.  
덧붙여, 아무래도 이전 글과 다른 데이터라, 별도로 글을 나누어 써본다.

In [ ]:
# 데이터 로드: 원본 데이터는 크기가 너무 크므로 압축된 데이터를 불러온다.
#   서울특별시 공공자전거 대여정보_201905.csv.gz (76.3MB)

from pathlib import Path

데이터_폴더 = Path('../data')
공공자전거_대여정보_201905 = 데이터_폴더 / '서울특별시 공공자전거 대여정보_201905.csv.gz'

df = pd.read_csv(공공자전거_대여정보_201905, encoding='cp949')
df.head()

In [ ]:
# 데이터 파일의 첫 줄에 헤더 정보가 없으므로, 컬럼 명을 직접 넣어주어야 한다.

df = pd.read_csv(공공자전거_대여정보_201905,
    encoding='cp949',
    names=['자전거번호',
           '대여일시', '대여대여소번호', '대여대여소명', '대여거치대',
           '반납일시', '반납대여소번호', '반납대여소명', '반납거치대',
           '이용시간', '이용거리'],
    parse_dates=['대여일시', '반납일시']
)
df.head()

In [ ]:
df.tail()

In [ ]:
# 대여일시와 반납일시에서 날짜(day), 요일(dayofweek), 시간(hour)을 분리하자.

df['대여일'] = df['대여일시'].dt.day
df['대여요일'] = df['대여일시'].dt.dayofweek
df['대여시간'] = df['대여일시'].dt.hour
df['반납시간'] = df['반납일시'].dt.hour

In [ ]:
df.info(memory_usage='deep')

In [ ]:
# [흠시] 대여소 번호는 있는데, 어떤 지역인지 모른다.....  
# 그래서 대여소 정보 데이터를 가져와서 합쳐야 한다.

공공자전거_대여소_정보_데이터 = 데이터_폴더 / '서울특별시 공공자전거 대여소 정보(19.12.9).xlsx'

rental = pd.read_excel(공공자전거_대여소_정보_데이터, skipfooter=1)
rental.head()

In [ ]:
rental = rental.astype({
    "대여소_구": "category",
    "대여소ID": "category",
    "위도": "category",
    "경도": "category"
})
rental.info()

In [ ]:
# 대여지역을 먼저 합쳐보자.

before_merged = len(df)

df = df.merge(rental[['대여소_구', '대여소ID', '위도', '경도']],
    left_on='대여대여소번호',
    right_on='대여소ID'
).rename(
    columns={
        "대여소_구": "대여지역",
        "위도": "대여대여소위도",
        "경도": "대여대여소경도"
    }
).drop(columns='대여소ID')
df.head()

In [ ]:
# 반납지역도 합쳐보자.

df = df.merge(rental[['대여소_구', '대여소ID', '위도', '경도']],
    left_on='반납대여소번호',
    right_on='대여소ID'
).rename(
    columns={
        "대여소_구": "반납지역",
        "위도": "반납대여소위도",
        "경도": "반납대여소경도"
    }
).drop(columns='대여소ID')
df.head()

In [ ]:
# [흠시] 2% 데이터를 잃었지만, 이 정도는 그냥 넘어가본다.

after_merged = len(df)
loss = before_merged - after_merged

print(df.shape)
print(f"{loss} loss. ({loss / before_merged * 100:.2f}%)")

In [ ]:
df.info(memory_usage='deep')

---
## 1. 어떤 요일, 시간에 이용량이 많았을까?

> [흠시] 가장 쉽게 떠오를 수 있는 질문이다.  
요일, 시간, 그리고 지역별로 하나씩 살펴보자.

### 1.1. 요일별 이용량

In [ ]:
use_by_dayofweek = df.groupby('대여요일').size()
use_by_dayofweek.index = "월 화 수 목 금 토 일".split()

ax = use_by_dayofweek.plot(
    kind='bar',
    rot=0,
    title="요일에 따른 이용량",
    figsize=(15, 5)
)
ax.set_frame_on(False)

> [흠시]  
> **평일보다 주말 이용량이 더 많은 것을 알 수 있다.**  
또, 평일 중엔 화요일 수요일이.  
주말에는 토요일 이용량이 많다.

In [ ]:
# 평일과 주말을 기준으로 비교해 보자.

mean_weekday = use_by_dayofweek.loc["월 화 수 목 금".split()].mean()
mean_weekend = use_by_dayofweek.loc["토 일".split()].mean()

ax = pd.Series(
    data=[mean_weekday, mean_weekend],
    index=["평일", "주말"]
).plot(
    kind='bar',
    rot=0,
    title="평일, 주말 평균 이용량 비교",
    figsize=(10, 5)
)
ax.set_frame_on(False)

In [ ]:
diff = (mean_weekend - mean_weekday) / mean_weekday * 100
print(f"주말이 평일대비 {diff:.1f}% 더 많다.")

### 1.2. 시간별 이용량

> [흠시] 평일과 주말에 따라 패턴이 다를 듯하여, 두 경우로 나누어 시각화 해보았다.

In [ ]:
# 평일

ax = pd.DataFrame(
    data={
        "대여량": df[df['대여요일'] < 5].groupby('대여시간').size() // 5,
        "반납량": df[df['대여요일'] < 5].groupby('반납시간').size() // 5
    }
).plot(
    kind='bar',
    rot=0,
    xlabel="시",
    title="시간에 따른 평균 이용량(평일)",
    figsize=(15, 5)
)
ax.legend(frameon=False)
ax.set_frame_on(False)

In [ ]:
# 주말

ax = pd.DataFrame(
    data={
        "대여량": df[df['대여요일'] >= 5].groupby('대여시간').size() // 2,
        "반납량": df[df['대여요일'] >= 5].groupby('반납시간').size() // 2
    }
).plot(
    kind='bar',
    rot=0,
    xlabel="시",
    title="시간에 따른 평균 이용량(주말)",
    figsize=(15, 5)
)
ax.legend(frameon=False)
ax.set_frame_on(False)

> [흠시] 다음과 같은 사실들을 알 수 있다.  
> - **평일의 경우, 대여/반납 시간대가 모두 8시와 18시에 몰려있다.** 주로 출퇴근 시간에 몰린듯 하다.
> - **주말의 경우, 밤시간대로 갈수록 몰린다.** 주로 18시 전후로 최고치를 찍는다. 해가 서서히 지는 시간대에 주로 타려고 하는 걸까?
> - 한편, 출근시간 제외하고, 일반적으로 **낮에는 대여량 > 반납량이고, 밤에는 그 반대**다.

> [흠시] 그런데, 위와 같은 패턴이 모든 지역에 다 동일하게 드러날까? 이것도 확인해보자.

### 1.3. 시간에 따른 지역구별 이용량

> [흠시] 시간에 따른 각 지역구별 이용량을 히트맵으로 살펴보자.  
주말은 일단 제외하고, 평일만 살펴보도록 한다.

In [ ]:
# 대여

_df = df[df['대여요일'] < 5].groupby(['대여지역', '대여시간']).size() // 5
_df

In [ ]:
pvt_table = _df.unstack()
pvt_table.head()

In [ ]:
_, ax = plt.subplots(figsize=(15, 10))
sns.heatmap(pvt_table,
    annot=True,
    fmt='d',
    cmap="Blues",
    cbar=False,
    linewidth=1,
    ax=ax
)
ax.set_title("지역별 시간에 따른 평균 대여량(평일)")
ax.set_ylabel("");

In [ ]:
# 반납

pvt_table = (
    df[df['대여요일'] < 5].groupby(['반납지역', '반납시간']).size() // 5
).unstack()

_, ax = plt.subplots(figsize=(15, 10))
sns.heatmap(pvt_table,
    annot=True,
    fmt='d',
    cmap="Oranges",
    cbar=False,
    linewidth=1,
    ax=ax
)
ax.set_title("지역별 시간에 따른 평균 반냡량(평일)")
ax.set_ylabel("");

> [흠시] 나란히 두어, 보기 좀 복잡할 수 있지만, 발견할 수 있는 가장 간단한 사실은,  
**예외 없이, 전 지역에서 동일한 시간대별 이용량 패턴**을 가진다는 것이다.

> [흠시] 특히, 출/퇴근 시간에 색이 진하게 그려져있는 것이 인상적인데, 여기서 다음과 같은 생각이 들었다.  
출근시간에 대여량이 반냡량보다 높은 지역은 어딜까?  
또, 반대로 반냡랑이 대여량보다 높은 지역은 어딜까?  
일반화 하면, 다음과 같은 질문이다.  
> > 출퇴근 시간에, 이용량 중 대여 혹은 반납이 집중되는 지역이 있을까?

> [흠시] 이를 알아보기 위해, 위 데이터에서 출/퇴근 시간의 피크인 8시, 18시만 가져와보자.  
그리고, 이용량 중, 대여와 반납의 비율을 살펴보자.

In [ ]:
# 대여

pvt_table = (
    df[df['대여요일'] < 5].groupby(['대여지역', '대여시간']).size() // 5
)
rent = pd.DataFrame(
    data=[pvt_table.xs(8, level=1), pvt_table.xs(18, level=1)],
    index=[8, 18]
)
rent

In [ ]:
# 반납

pvt_table = (
    df[df['대여요일'] < 5].groupby(['반납지역', '반납시간']).size() //5
)
rtrn = pd.DataFrame(
    data = [pvt_table.xs(8, level=1), pvt_table.xs(18, level=1)],
    index=[8, 18]
)
rtrn

In [ ]:
total = rent + rtrn
total

In [ ]:
rent = rent / total
rtrn = rtrn / total

In [ ]:
morning_diff = pd.DataFrame(
    [rent.loc[8], rtrn.loc[8]],
    index=['대여', '반납']
).T.sort_values('대여')
morning_diff

In [ ]:
dinner_diff = pd.DataFrame(
    [rent.loc[18], rtrn.loc[18]],
    index=['대여', '반납']
).T.sort_values('대여')
dinner_diff

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 10))

for i, (diff, time_name) in enumerate(
    zip([morning_diff, dinner_diff], ["출근시간(8시)", "퇴근시간(18시)"])
):
    ax = diff.plot(
        kind='barh',
        color=['C0', 'orange'],
        stacked=True,
        title=f"{time_name}에 사용량 비율",
        ax=axes[i]
    )
    
    for p in ax.patches: 
        left, bottom, width, height = p.get_bbox().bounds 
        ax.annotate(f"{width * 100:.1f}%", xy=(left+width/2, bottom+height/2), ha='center', va='center')

    ax.set_xticks([])
    ax.set_frame_on(False)

axes[0].get_legend().remove()
axes[1].legend(loc='center left', bbox_to_anchor=(0.97, 0.5), frameon=False)
fig.tight_layout()

> [흠시] 출근시간엔, **서대문구**가 대여량이 반납량보다 **전체의 약 27%** 정도 많았다.  
퇴근시간엔, **금천구**가 대여량이 반납량보다 **전체의 20%** 정도 많았다.

> [흠시] 하지만 위의 두 플롯은 y축의 순서가 달라, 지역별로 출퇴근 시간의 이용량을 비교하기 힘들다.  
이를 위해 아래와 같이 다시 시각해해보자.  
반납량 = 이용량 - 대여량이므로, 보기 쉽게 대여량만 시각화해본다.

In [ ]:
morning_dinner_diff = pd.DataFrame(
    data=[morning_diff['대여'], dinner_diff['대여']],
    index=['출근시간', '퇴근시간']
).T.sort_values('출근시간', ascending=False)

ax = morning_dinner_diff.plot(
    kind='bar',
    rot=0,
    color=['royalblue', 'midnightblue'],
    title="출퇴근 시간, 지역별 사용량 중 대여비율",
    figsize=(15, 5)
)
ax.legend(frameon=False)
ax.set_frame_on(False)

> [흠시] 잘보면, 출근시간 막대그래프는 점점 내려가는데, 퇴근시간 막대그래프는 점점 올라가는 추세다.  
즉, **출근시간에 대여비중이 높았던 지역은, 퇴근시간에는 반납비중이 높다. (상관계수가 0.8로 나온다.)**  
이를 쉽게 해석하면, 이용자들이 거주지 -> 근무지로 이동하는 것으로 상상해볼 수 있다.  
**즉, 거주지역은 출근시간 대여량이 높고, 근무지역은 퇴근시간 대여량이 높은 것이다.**  
어느정도 상식선의 이야기다.

> [흠시] 이러한 관점에서 보면, 각 지역에서 타지역으로가는 유출/유입에 대해서도 생각해볼 수 있는데, 이에 관한 이야기는 뒤에 적어보도록 하겠다.

---
## 2. 요일별로 이용량이 높은 지역은 고정되어있을까?

> [흠시] 위 히트맵에서, 시간대별로 모든 지역이 같은 이용량 패턴을 가지고 있음을 확인했다.  
또한, 각 지역들의 이용량을 수치적으로 확인하고, 어디가 많고 적은지 알 수 있었다.  
그렇다면, 이용량이 많은 지역들은 일주일 내내 많을까?  
즉, 이용량이 많은 지역은 고정적일까?

### 2.1. 요일별, 지역의 이용량 순위

In [ ]:
# 요일별 대여지역의 이용량

use_by_region = df.pivot_table(
    index='대여지역',
    columns='대여요일',
    aggfunc='size'
) + df.pivot_table(
    index='반납지역',
    columns='대여요일',
    aggfunc='size'
)
use_by_region

In [ ]:
# 요일별 대여지역의 이용량 순위

use_by_region_rank = (use_by_region
    .rank()
    .sort_values(by=0, ascending=False)
)
use_by_region_rank

In [ ]:
# 요일에 따라서 지역별 사용량 순위가 달라질까?

xticks = "월 화 수 목 금 토 일".split()
yticks = list(use_by_region_rank.index)

ax = use_by_region_rank.T.plot(
    style='.-',
    legend=False,
    xlabel="",
    xticks=range(len(xticks)),
    yticks=range(len(yticks)),
    title="요일별 사용량 높은 지역 순위",
    figsize=(15, 10))
ax.set_xticklabels(xticks)
ax.set_yticklabels(reversed(yticks))
ax.set_frame_on(False)

> [흠시] 비교적 순위 변동이 없다. 바뀌어봐야 대부분 1~2등수 바뀌는 수준이다.  
즉, **요일에 상관없이, 이용량이 많은 지역은 고정적이라고 볼 수 있다.**

In [ ]:
# [흠시] 평일이든, 주말이든, 이용률이 높은 지역은 고정되어있다. 차이의 변화도 그다지 없다.

weekday = [0, 1, 2, 3, 4]
weekend = [5, 6]

fig, axes = plt.subplots(1, 2, figsize=(15, 10))

for i, (dayofweek, name) in enumerate(
    zip([weekday, weekend], ["평일", "주말"])
):
    rental_by_region = df[df['대여요일'].isin(dayofweek)].groupby('대여지역').size()
    return_by_region = df[df['대여요일'].isin(dayofweek)].groupby('반납지역').size()

    use_by_region = pd.DataFrame(
        data={
            "대여": rental_by_region,
            "반납": return_by_region
        }
    ).sort_values("대여", ascending=True)

    ax = use_by_region.plot(
        kind='barh',
        rot=0,
        title=f"지역별 이용량({name})",
        ax=axes[i]
    )
    ax.set_frame_on(False)

axes[0].get_legend().remove()
axes[1].legend(loc="right", frameon=False)
fig.tight_layout()

---
## 3. 평균 이용거리, 이용시간이 높은 지역은 어딜까?

> [흠시] 어떤 지역에 평균 이용거리, 이용시간이 높다는 말은, 장거리 이동 또는 오래 이용하는 이용자가 많다는 뜻이다.  
이용거리부터 하나씩 살펴보자.

### 3.1. 이동거리

> [흠시] 먼저 이용거리 전체 분포를 boxplot 과 distplot 으로 보자.

In [ ]:
# 전체 분포

distance = df['이용거리']
distance

In [ ]:
# 0인 값 제외
distance = distance[distance != 0]
distance

In [ ]:
def draw_box_distplot(series, title, xlabel, axvline=False, color='C0', bins=100):
    # Cut the window in 2 parts
    f, (ax_box, ax_hist) = plt.subplots(
        nrows=2,
        sharex=True,
        figsize=(15, 5),
        gridspec_kw={"height_ratios": (.15, .85)}
    )

    # Add a graph in each part
    sns.boxplot(x=series, ax=ax_box, boxprops={'alpha': 0.6}, color=color)
    sns.histplot(data=series, stat='density', kde=True, linewidth=0, ax=ax_hist, color=color, bins=bins)

    # Remove x axis name for the boxplot
    ax_box.set_xlabel("")
    ax_box.set_title(title)
    ax_box.set_frame_on(False)

    ax_hist.set_xlabel(xlabel)
    ax_hist.set_ylabel("")
    ax_hist.set_frame_on(False)
    
    if axvline:
        ax_hist.axvline(series.mean(), color='green')
        ax_hist.axvline(series.value_counts().idxmax(), color='red')
        ax_hist.axvline(series.median(), color='blue')
    
    fig.tight_layout()
    
draw_box_distplot(distance, "이용거리(m) 분포", "이용거리(m)", color='purple')

> [흠시] 꽤 많은 극단치들이 있어서, 굉장히 skewed 되어있다.  
이대로도 의미가 있기는 하겠지만, 일반적인 상황에 대해 분석하기가 힘드므로, 극단치들을 잘라내고 다시 그려보자.

In [ ]:
def remove_outlier(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1 # Interquartile range
    fence_low  = q1 - 1.5 * iqr
    fence_high = q3 + 1.5 * iqr
    series_out = series.loc[(series > fence_low) & (series < fence_high)]
    return series_out

distance = remove_outlier(distance)
draw_box_distplot(distance, "이용거리(m) 분포", "이용거리(m)", axvline=True, color='purple')

In [ ]:
# 빨간선은 최빈값, 파란선은 중간값, 초록선은 평균을 나타낸다.

print(f"최빈값: {distance.value_counts().idxmax():.0f}m")
print(f"중간값: {distance.median():.0f}m")
print(f"절사평균(5~95%): {distance.mean():.0f}m")

> [흠시] 그럼에도 skewed 되어있기는한데, 중간값은 2400m다.  
최빈값은 910m로, 일반적인 라이트 유저(?)라면 이 정도 거리를 타는 듯 하다.

> [흠시] 지역별로 살펴보면,

In [ ]:
# 지역별 평균 이용 거리

distance = df[['대여지역', '이용거리']]
distance = distance[distance['이용거리'] != 0] # 0인 값 제외
distance.head()

In [ ]:
medians = (distance
    .groupby('대여지역')
    .median()
    .squeeze()
    .sort_values(ascending=False)
)
medians.head()

In [ ]:
distance_df_total = (distance
    .groupby('대여지역')
    .apply(lambda df: remove_outlier(df['이용거리']))
    .reset_index(level=0)
)
distance_df_total.head()

In [ ]:
# [흠시] 중간값을 기준으로 정렬했다.

_, ax = plt.subplots(figsize=(15, 5))
sns.violinplot(
    x='대여지역',
    y='이용거리',
    data=distance_df_total,
    order=medians.index,
    palette="Purples_r",
    ax=ax
)
ax.set_frame_on(False)

In [ ]:
_, ax = plt.subplots(figsize=(15, 5))
sns.barplot(
    x='대여지역',
    y='이용거리',
    data=medians.reset_index(),
    palette="Purples_r",
    ax=ax
)
ax.set_xlabel("")
ax.set_ylabel("")
ax.set_title("지역별 이용거리(m) 중간값")
ax.set_frame_on(False)

> [흠시]  
> - 일단 모든 지역에서 분포가 skewed 하다.
> - 평균 이용거리가 큰 상위 지역들은, 중간값도 크지만, 극단치도 컸다. 일반적으로 다른지역들 보다 헤비하다(?)고 볼 수 있겠다.
> - 양천구의 경우, 중간값은 낮은편이지만, 꽤 높은 극단치들이 있다. 일부 사용자들이 굉장히 헤비하게 이용한다고 볼 수 있겠다.

In [ ]:
# 나중을 위해 보관

distance_medians = medians

### 3.2. 이용시간

> [흠시] 위 방법과 마찬가지로 살펴보면,

In [ ]:
# 전체 분포

time = df['이용시간']
time = time[time != 0]

draw_box_distplot(time, "이용시간(분) 분포", "이용시간(분)", color='brown')

In [ ]:
time = remove_outlier(time)
draw_box_distplot(time, "이용시간(분) 분포", "이용시간(분)", axvline=True, color='brown', bins=74)

In [ ]:
print(f"최빈값: {time.value_counts().idxmax()}분")
print(f"중간값: {time.median():.0f}분")
print(f"절사평균(5~95%): {time.mean():.0f}분")

> [흠시] 이용거리 분포와 거의 동일하다.  
최빈값은 6분이고, 중간값은 17분이다.

In [ ]:
time = df[['대여지역', '이용시간']]
time = time[time['이용시간'] != 0]

usetime_df_total = (time
    .groupby('대여지역')
    .apply(lambda df: remove_outlier(df['이용시간']))
    .reset_index(level=0)
)
usetime_df_total.head()

In [ ]:
medians = (usetime_df_total
    .groupby('대여지역')
    .median()
    .squeeze()
    .sort_values(ascending=False)
)
medians.head()

In [ ]:
_, ax = plt.subplots(figsize=(15, 5))
sns.violinplot(
    x='대여지역',
    y='이용시간',
    data=usetime_df_total,
    order=medians.index,
    palette="YlOrBr_r",
    ax=ax
)
ax.set_xlabel("")
ax.set_ylabel("이용시간 (분)")
ax.set_frame_on(False)

In [ ]:
_, ax = plt.subplots(figsize=(15, 5))
sns.barplot(
    x='대여지역',
    y='이용시간',
    data=medians.reset_index(),
    palette="YlOrBr_r",
    ax=ax
)
ax.set_xlabel("")
ax.set_ylabel("")
ax.set_title("지역별 이용시간(분) 중간값")
ax.set_frame_on(False)

> [흠시] 위와 비슷하다.  
다만 양천구에 길었던 꼬리가 없어졌다.  
이용거리가 길었지만 이용시간은 짧았던 이용자... 엄청난 속도로 따릉이를 탄 사람이 양천구에 있나보다...

In [ ]:
# 나중을 위해 보관

usetime_medians = medians

> [흠시] 지도에 위 내용을 표현하면,

In [ ]:
# 왼쪽(보라색)은 이용거리, 오른쪽(갈색)은 이용시간을 나타낸다.

import json, folium, folium.plugins

geo_path = '../data/seoul_municipalities_geo_simple.json'
with open(geo_path, encoding='utf-8') as fp:
    geo_str = json.load(fp)

bike_map = folium.plugins.DualMap(
    location=[37.541, 126.986],
    zoom_start=10,
    tiles='CartoDB positron',
    zoom_control=False
)
folium.Choropleth(geo_str,
    data=distance_medians,
    key_on='feature.properties.SIG_KOR_NM', 
    fill_color='Purples',
    line_color='grey',
    line_opacity=0.5
).add_to(bike_map.m1)
folium.Choropleth(geo_str,
    data=usetime_medians,
    key_on='feature.properties.SIG_KOR_NM', 
    fill_color='YlOrBr',
    line_color='grey',
    line_opacity=0.5
).add_to(bike_map.m2)
bike_map

> [흠시]  
> - **평균 이용거리, 이용시간이 높은 지역은 용산구다.**
> - 이용거리 - 이용시간은 높은 선형관계다(상관계수 0.94). 멀리가려면, 오래 이용해야하니, 상식적으로 당연한 결과이긴 하다.

---
## 4. 유출/유입이 많은 지역은 어딜까?

> [흠시] 이제 드디어, 유출/유입에 대한 이야기를 써볼까 한다.  
유출/유입량이란, 어떤 지역에서 다른 지역으로 넘어가거나 넘어오는 따릉이 이용량을 말한다.  
즉, 지역구간 이동하는 트래픽에 대해 알아볼 수 있다.

> [흠시] 먼저, 각 지역별, 사용량의 스케일이 다르므로, 절대량이 아닌 상대비율로 지역간 비교를 해본다.   
여기서는 유출/유입 비율을 사용할 건데, 아래와 같이 정의한다.  
>> `유출비율 = A지역 대여 후 타 지역에 반납한 량 / A지역 총 대여량`  
`유입비율 = 타 지역에서 대여 후 A지역에 반납한 량 / A지역 총 반납량`

> [흠시] 즉, 한마디로 말해, **해당 지역의 총 대여(반납)량 중에 몇 %가 타지역에 반납(대여)되었는지**를 말하는 것이다.

### 4.1. 유출/유입비율 큰 순으로 보기

> [흠시] 가장 먼저 생각해볼 수 있는 것은, 정말 말그대로 유출비율과 유입비율이 큰 지역을 살펴보는 것이다.  
바로 지도로 시각화 해보자.

In [ ]:
# 유출비율

outflow_by_region = df[['대여지역', '반납지역']].groupby('대여지역').apply(
    lambda df: (df['대여지역'] != df['반납지역']).sum() / len(df) * 100
).sort_values(ascending=False)

outflow_by_region.name = '유출비율'
outflow_by_region

In [ ]:
_, ax = plt.subplots(figsize=(15, 5))
sns.barplot(
    x='대여지역',
    y='유출비율',
    data=outflow_by_region.reset_index(),
    order=outflow_by_region.index,
    palette='Reds_r'
)
ax.set_title("지역별 유출비율(%)")
ax.set_frame_on(False)

In [ ]:
bike_map = folium.Map(
    location=[37.541, 126.986],
    zoom_start=10.8,
    tiles='CartoDB positron',
    zoom_control=False
)
folium.Choropleth(geo_str,
    data=outflow_by_region,
    key_on='feature.properties.SIG_KOR_NM', 
    fill_color='Reds',
    line_color='grey',
    line_opacity=0.5
).add_to(bike_map)
bike_map

In [ ]:
# 유입비율

inflow_by_region = df[['대여지역', '반납지역']].groupby('반납지역').apply(
    lambda df: (df['대여지역'] != df['반납지역']).sum() / len(df) * 100
).sort_values(ascending=False)

inflow_by_region.name = '유입비율'
inflow_by_region

In [ ]:
_, ax = plt.subplots(figsize=(15, 5))
sns.barplot(
    x='반납지역',
    y='유입비율',
    data=inflow_by_region.reset_index(),
    order=inflow_by_region.index,
    palette='Greens_r'
)
ax.set_title("지역별 유입비율(%)")
ax.set_frame_on(False)

In [ ]:
bike_map = folium.Map(
    location=[37.541, 126.986],
    zoom_start=10.8,
    tiles='CartoDB positron',
    zoom_control=False
)
folium.Choropleth(geo_str,
    data=inflow_by_region,
    key_on='feature.properties.SIG_KOR_NM', 
    fill_color='Greens',
    line_color='grey',
    line_opacity=0.5
).add_to(bike_map)
bike_map

In [ ]:
# [흠시] 유출이 많을수록, 유입이 많은걸까?

inoutflow_by_region = pd.DataFrame(
    data={'유출비율': outflow_by_region, '유입비율': inflow_by_region}
).sort_values(by='유출비율', ascending=False)
inoutflow_by_region.head()

In [ ]:
ax = inoutflow_by_region.plot(
    kind='bar',
    rot=0,
    color=['C3', 'C2'], # red, green
    figsize=(15, 5)
)
ax.set_title("지역별 유출/유입비율(%)")
ax.legend(frameon=False)
ax.set_frame_on(False)

In [ ]:
# 왼쪽(빨간색)은 유출비율, 오른쪽(초록색)은 유입비율을 나타낸다.

bike_map = folium.plugins.DualMap(
    location=[37.541, 126.986],
    zoom_start=10.8,
    tiles='CartoDB positron',
    zoom_control=False
)
folium.Choropleth(geo_str,
    data=outflow_by_region,
    key_on='feature.properties.SIG_KOR_NM', 
    fill_color='Reds',
    line_color='grey'
).add_to(bike_map.m1)
folium.Choropleth(geo_str,
    data=inflow_by_region,
    key_on='feature.properties.SIG_KOR_NM', 
    fill_color='Greens',
    line_color='grey'
).add_to(bike_map.m2)
bike_map

In [ ]:
inoutflow_by_region.corr()

> [흠시]  
> - 한 눈에봐도 중구가 눈에 띈다. 또한, 서울 안쪽지역, 즉 **중심부가 확실히 유입/유출 비율이 높다.**
> - **유출비율과 유입비율은 거의 동일한 분포**를 가진다. 즉 유출비율이 높은 지역은 유입비율도 높다. (상관계수 0.98)

### 4.2. 시간을 더해서 살펴보기

> [흠시] 이번엔 시간의 개념을 더해서 살펴보자. (평일 기준)

In [ ]:
# 유출
# [흠시] 조금 더 상세히 알아보기 위해, 각 지역이, 시간대별 대여량 중 외부 지역으로 유출하는 비율을 알아보자.

df_weekday = df[df['대여요일'] < 5]

outflow_by_region_weekday = df_weekday.groupby('대여지역').apply(
    lambda x: x[x['대여지역'] != x['반납지역']].groupby('대여시간').size() \
               / x.groupby('대여시간').size() * 100
)
outflow_by_region_weekday.head()

In [ ]:
ax = outflow_by_region_weekday.T.plot(
    style='.-',
    rot=0,
    xticks=range(24),
    title="지역별, 각 시간의 유출비율 (%, 평일)",
    figsize=(15, 10)
)
ax.set_frame_on(False)

In [ ]:
_, ax = plt.subplots(figsize=(15,10))
sns.heatmap(
    data=outflow_by_region_weekday,
    vmin=0,
    vmax=100,
    cmap='Reds',
    annot=True,
    fmt=".1f",
    linewidth=1,
    cbar=False,
    square=False,
    ax=ax
)
ax.set_title("지역별, 각 시간의 유출비율 (%, 평일)");

In [ ]:
# 유입

inflow_by_region_weekday = df_weekday.groupby('반납지역').apply(
    lambda df: df[df['대여지역'] != df['반납지역']].groupby('반납시간').size() \
               / df.groupby('반납시간').size() * 100
)
inflow_by_region_weekday.head()

In [ ]:
ax = inflow_by_region_weekday.T.plot(
    style='.-',
    rot=0,
    xticks=range(24),
    title="지역별, 각 시간의 유입비율 (%, 평일)",
    figsize=(15, 10)
)
ax.set_frame_on(False)

In [ ]:
_, ax = plt.subplots(figsize=(15,10))
sns.heatmap(
    data=inflow_by_region_weekday,
    vmin=0,
    vmax=100,
    cmap='Greens',
    annot=True,
    fmt=".1f",
    linewidth=1,
    cbar=False,
    square=False,
    ax=ax
)
ax.set_title("지역별, 각 시간의 유입비율 (%, 평일)");

> [흠시] 시각화해서 보니, 시간에 따른 **유출비율과 유입비율 패턴이 조금 다르다!**  
얼핏봐도, 위 두 히트맵에의 색 분포가 조금 다름을 느낄 수 있을 것이다.  
자연스레 이런 생각이 든다.  
> > 시간에 따라, 유출 혹은 유입비율이 비슷한 패턴을 가지는 지역들이 있을까?

> [흠시] 같은 시간대에, 유출비율과 유입비율을 동시에 확인해야 하므로, **측정값 = (유입비율에서 유출비율을 뺀 값)**을 지표로 삼자.  
즉, 측정값 > 0 이면, 해당 시간에 유입비율이 더 많다는 것이고, 측정값 < 0 이면, 유출비율이 더 많다는 것이다. 

> [흠시] 이렇게 시간에 따라 측정값 패턴이 비슷한 지역들끼리 clustermap 으로 묶어보자.

In [ ]:
inout_ratio = inflow_by_region_weekday - outflow_by_region_weekday
inout_ratio.head()

#### [Plot a matrix dataset as a hierarchically-clustered heatmap](https://seaborn.pydata.org/generated/seaborn.clustermap.html)

```python
seaborn.clustermap(data, pivot_kws=None, method='average', metric='euclidean', z_score=None, standard_scale=None, figsize=(10, 10), cbar_kws=None, row_cluster=True, col_cluster=True, row_linkage=None, col_linkage=None, row_colors=None, col_colors=None, mask=None, dendrogram_ratio=0.2, colors_ratio=0.03, cbar_pos=(0.02, 0.8, 0.05, 0.18), tree_kws=None, **kwargs)
```

- The returned object has a `savefig` method that should be used if you want to save the figure object without clipping the dendrograms. 
- To access the reordered row indices, use: `clustergrid.dendrogram_row.reordered_ind`
- Column indices, use: `clustergrid.dendrogram_col.reordered_ind`

In [ ]:
clustergrid = sns.clustermap(inout_ratio.T.corr(),
    cmap='Blues',
    vmin=-1,
    vmax=1,
    cbar=True
)

> [흠시] 어느정도 뚜렷히 보인다.  
시각화 이후 클러스터에 A, B, C 라는 이름을 주었다.

> [흠시] 이제 측정값을 다시 시간에 따라서 살펴보면,

In [ ]:
reordered_ind = clustergrid.dendrogram_row.reordered_ind
inout_ratio.index[reordered_ind]

In [ ]:
inout_ratio = inout_ratio.reindex(inout_ratio.index[reordered_ind])
# or inout_ratio.reset_index().reindex(reordered_ind).set_index('반납지역')
inout_ratio.head()

In [ ]:
# [흠시] 시간에 따른 각 지역별 (유입 - 유출비율) 히트맵. 빨간색은 유출이 압도함을, 초록색은 유입이 압도함을 의미한다.

_, ax = plt.subplots(figsize=(15, 10))
sns.heatmap(inout_ratio,
    vmin=-32,
    vmax=32,
    square=False,
    annot=True,
    fmt=".1f",
    cmap='RdYlGn', 
    cbar=False,
    ax=ax
)
ax.set_title("지역별, 각 시간의 유입비율-유출비율(%, 평일)")
ax.set_ylabel("");

![](k.kakaocdn.net_dn_bhggpc_btqwR3ZVEbs_tDBj9QjXSL0IPRR4USH76K_img.png)

> [흠시] 실제로 의미있게 클러스터링 된 듯하다.

> [흠시] **A 클러스터**에 속한 지역들은 출근시간에 유입이 많고, 퇴근시간에 유출이 많은 지역이다. 아무래도, 상업지역이지 않을까 싶다.  
**B 클러스터**에 속한 지역들은 A 클러스터와 정반대의 양상이다. 출근하는 사람들이 많이 살고있는 주거지역일 듯 싶다.  
**C 클러스터**는 A, B 두 클러스터에 속하지 않는 지역들이다. 다른 두 클러스터에 비해, 패턴이 뚜렷치 않다.

> [흠시] 한편, 몇몇 눈에띄는 이상치(?) 들이 보이는데,  
> - 새벽 1-3시에 중구에 눈에 띄게 유출비율이 높다는 점.
> - 새벽 5시에 금천구와 구로구에 유입비율이 유독 높다는 점.

> [흠시] 정도가 보인다. 이유는 잘 모르겠다.  
또 한편으로, 출퇴근 시간대를 잘보면, 유입비율이 높은 시간대는 유출비율이 높은 시간대보다 한 시간씩 밀려있는 것을 볼 수 있다.

### 4.3. 사람들이 주로 출/퇴근하는 지역은 어딜까?

> [흠시] 위 히트맵에서, 출퇴근 시간대의 유출-유입비율을 이번엔 지도로 시각화 해보자.  
출근시간은 7시\~10시, 퇴근시간은 17\~20시 사이로 가정하였다.

In [ ]:
# 출근시간대

mean_inout_morning = inout_ratio.iloc[:, 7:10].mean(axis=1)
mean_inout_morning

In [ ]:
# 퇴근시간대

mean_inout_night = inout_ratio.iloc[:, 17:20].mean(axis=1)
mean_inout_night

In [ ]:
# 왼쪽은 출근시간대, 오른쪽은 퇴근시간대를 나타낸다.

bike_map = folium.plugins.DualMap(
    location=[37.541, 126.986],
    zoom_start=10,
    tiles='CartoDB positron',
    zoom_control=False
)
folium.Choropleth(geo_str,
    data=mean_inout_morning,
    key_on='feature.properties.SIG_KOR_NM', 
    fill_color='RdYlGn',
    line_color='grey'
).add_to(bike_map.m1)
folium.Choropleth(geo_str,
    data=mean_inout_night,
    key_on='feature.properties.SIG_KOR_NM', 
    fill_color='RdYlGn',
    line_color='grey'
).add_to(bike_map.m2)
bike_map

> [흠시] 출근시간대 유출비율이 높았던 지역이, 퇴근시간대에는 유입비율이 더 높은 것을 알 수 있다.

### 4.4. 출/퇴근시간대에 유입/유출이 활발한 지역은 어딜까?

> [흠시] 이번에 궁금한 것은, 위와 같은 유입-유출의 변화가 가장 드라마틱한 지역은 어딜까? 이다.  
예를 들어, 출근 시간대에 유입비율이 아주 높고, 퇴근시간에는 유출비율이 아주 높은 지역은 유입-유출의 변화가 드라마틱한 지역이라고 할 수 있다.  
수식으로는, 절대값(유입비율) + 절대값(유출비율) 의 값인데, 이 값이 제일 큰 지역이 어딘지 살펴보자.

In [ ]:
diff_inout_abs = (abs(mean_inout_morning) + abs(mean_inout_night)).sort_values()
diff_inout_abs

In [ ]:
# 차이값이 큰 순으로 정렬

ax = diff_inout_abs.plot(
    kind='barh',
    rot=0,
    color='grey',
    xlabel="",
    ylabel="",
    title="출/퇴근 시간에 유입-유출 비율 차이(절대값)",
    figsize=(15, 10)
)
ax.set_frame_on(False)
for p in ax.patches: 
    x, y, width, height = p.get_bbox().bounds
    ax.text(width+0.3, y+height/2, f"{width:.1f}%", va='center')

> [흠시] **금천구가 32.8% 차이로 가장 크다!**  
다음은 은평구, 관악구 순이 되겠다.

> [흠시] 이를 보기 쉽게 지도로 시각화 하면, 아래와 같다.

In [ ]:
# 색이 진할수록 값이 크다.

bike_map = folium.Map(
    location=[37.541, 126.986],
    zoom_start=10.8,
    tiles='CartoDB positron',
    zoom_control=False
)
folium.Choropleth(geo_str,
    data=diff_inout_abs,
    key_on='feature.properties.SIG_KOR_NM', 
    fill_color='Greys'
).add_to(bike_map)
bike_map

---
## 정리

모든 내용을 다 정리하기보단, 따릉이 이용량에 특징있는 지역구만 정리해보면, 다음과 같다.

1. 일반적으로 이용량이 많은 지역: 마포, 영등포, 송파구
2. 평균 이용거리 및 이용시간이 큰 지역: 용산구
3. 유출비율과 유입비율이 높은 지역: 중구
4. 출/퇴근 시간에 유입/유출이 드라마틱하게 차이나는 지역: 금천구, 은평구